# run length and adapt vs. average risk adjustment

- how long does hawk/dove multi risk attitude take to converge with the new logic?
  - how many do not converge ?
- what difference does it make if we use adapt or average risk adjustment strategy?

In [143]:
import polars as pl

df = pl.scan_csv("../../data/hawkdovemulti/riskadjust/dist-uniform/*.csv").collect()

In [144]:
total_runs = len(df)

print(f"Analyzing {total_runs} runs")

Analyzing 1000 runs


## simulation run length

They either finished very quickly (~50 steps) or never finished)

In [145]:
df["Step"].describe()

statistic,value
str,f64
"""count""",1000.0
"""null_count""",0.0
"""mean""",2904.7
"""std""",2131.600767
"""min""",60.0
"""25%""",750.0
"""50%""",3500.0
"""75%""",5500.0
"""max""",5500.0


In [146]:
df["Step"].plot.hist()

:Histogram   [Step]   (Step_count)

In [147]:
# what about those that converged?

converged = df.filter(pl.col("status") == "converged")

In [148]:
converged["Step"].plot.hist()

:Histogram   [Step]   (Step_count)

### what % converged?

In [149]:
status_totals = df["status"].value_counts()
status_totals

status,count
str,u32
"""running""",858
"""converged""",142


In [150]:
converg_total = status_totals.filter(status_totals["status"] == "converged")["count"][0]

print(f"{converg_total} runs out of {total_runs}; {converg_total/total_runs*100:.2f}% complete")

142 runs out of 1000; 14.20% complete


### risk adjustment (adopt / average)

hypothesis: adjustment strategy does not have a significant impact on the final result, only affects how long it takes to get there

In [151]:
from scipy import stats


df_riskadjust = converged.clone()

# TODO: make reusable functions for annotating data

for i in range(0, 10):
    # calculate new series based on existing 
    pct_risk_category = df_riskadjust.select(pl.col(f"total_r{i}") / pl.col("total_agents"))
    # add new column to the dataframe
    df_riskadjust = df_riskadjust.with_columns(pl.Series(name=f"pct_r{i}", values=pct_risk_category))

df_riskadjust = df_riskadjust.with_columns(
    pl.Series('pct_risk_inclined', values=df_riskadjust.select((pl.col("total_r0") + pl.col("total_r1") + pl.col("total_r2")) / pl.col("total_agents"))),
    pl.Series('pct_risk_moderate', values=df_riskadjust.select((pl.col("total_r3") + pl.col("total_r4") + pl.col("total_r5") + pl.col("total_r6")) / pl.col("total_agents"))),
    pl.Series('pct_risk_avoidant', values=df_riskadjust.select((pl.col("total_r7") + pl.col("total_r8") + pl.col("total_r9")) / pl.col("total_agents")))
)

df_riskadjust = df_riskadjust.with_columns(pl.Series('risk_attitude_mean', values=df_riskadjust.select(
        (pl.col("total_r1") + pl.col("total_r2")*2 + pl.col("total_r3")*3 + pl.col("total_r4")*4 + pl.col("total_r5")*5 + pl.col("total_r6")*6 + pl.col("total_r7")*7 + pl.col("total_r8")*8 + pl.col("total_r9")*9)  
         / pl.col("total_agents"))))


df_adopt = df_riskadjust.filter((pl.col("risk_adjustment") == "adopt"))
df_avg = df_riskadjust.filter((pl.col("risk_adjustment") == "average"))

print(f"adopt: {len(df_adopt):,} rows")
print(f"average: {len(df_avg):,} rows")


adopt: 140 rows
average: 2 rows


In [152]:

maxlen = min(len(df_adopt), len(df_avg))

stats.ttest_rel(df_adopt.select("pct_risk_inclined")[:maxlen], df_avg.select("pct_risk_inclined")[:maxlen])

TtestResult(statistic=array([0.58333333]), pvalue=array([0.66381736]), df=array([1]))

In [153]:
df_riskadjust.plot.box("Step", by='risk_adjustment')

:BoxWhisker   [risk_adjustment]   (Step)

In [154]:
from simulatingrisk.hawkdovemulti import analysis_utils
import importlib
importlib.reload(analysis_utils)

# df_adopt, df_average

adopt_chart = analysis_utils.graph_population_risk_category(
    analysis_utils.groupby_population_risk_category(df_adopt)
).properties(title="risk adjust: adopt")
                                                                                                                                  
average_chart = analysis_utils.graph_population_risk_category(
    analysis_utils.groupby_population_risk_category(df_avg)
).properties(title="risk adjust: average")

(adopt_chart | average_chart).properties(title="distribution of population category by run").resolve_scale(y='shared')

alt.HConcatChart(...)

In [155]:
not_converged = df.filter(pl.col("status") == "running")
not_converged.shape

(858, 23)

In [158]:
not_converged["num_agents_risk_changed"].describe()

statistic,value
str,f64
"""count""",858.0
"""null_count""",0.0
"""mean""",10.850816
"""std""",7.569892
"""min""",1.0
"""25%""",4.0
"""50%""",10.0
"""75%""",16.0
"""max""",35.0


In [160]:
not_converged.plot.box("num_agents_risk_changed")

:BoxWhisker   (num_agents_risk_changed)